In [ ]:
import os, time, sys
from pathlib import Path
import sqlite3
import cv2
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D

ROOT = Path().absolute().parent
GROUND_TRUTH_PATH = Path(f"{ROOT}/datasets/DS2_Bode_Museum_Raw")
COMPARISON_PATHS = [
    Path(f"{ROOT}/datasets/DS3_Bode_Museum_Filter1")
]
GROUND_TRUTH_DB_PATH = f"{GROUND_TRUTH_PATH}/database.db"

if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

COLMAP_PY_SCRIPTS_PATH = f"{ROOT}/third_party/colmap/scripts/python"
if COLMAP_PY_SCRIPTS_PATH not in sys.path:
    sys.path.append(COLMAP_PY_SCRIPTS_PATH)

In [ ]:
scr = f"{ROOT}/third_party/colmap/scripts/python/export_inlier_matches.py"
OUT_PATH = f"{GROUND_TRUTH_PATH}/inlier_matches.txt"
! python {scr} --database_path "{GROUND_TRUTH_DB_PATH}" --output_path "{OUT_PATH}" --min_num_matches 15

In [ ]:
scr = f"{ROOT}/third_party/colmap/scripts/python/export_inlier_pairs.py"
OUT_PATH = f"{GROUND_TRUTH_PATH}/inlier_pairs.txt"
! python {scr} --database_path "{GROUND_TRUTH_DB_PATH}" --match_list_path "{OUT_PATH}" --min_num_matches 15

In [ ]:
from read_write_model import read_model, qvec2rotmat

In [ ]:
# init Colmap db reader functions from https://github.com/colmap/colmap/blob/main/scripts/python/database.py#L412
MAX_IMAGE_ID = 2 ** 31 - 1

# Note that COLMAP supports:
#      - 2D keypoints: (x, y)
#      - 4D keypoints: (x, y, theta, scale)
#      - 6D affine keypoints: (x, y, a_11, a_12, a_21, a_22)

def get_image(cursor, img_id):
    cursor.execute(f"SELECT i.name, c.width, c.height FROM images as i, cameras as c WHERE c.camera_id = i.camera_id AND i.image_id = {img_id}")
    image_data = cursor.fetchone()
    image_name, width, height = image_data
    return cv2.imread(f"{GROUND_TRUTH_PATH}/images_raw/{image_name}")

def pair_id_to_image_ids(pair_id):
    image_id2 = pair_id % MAX_IMAGE_ID
    image_id1 = (pair_id - image_id2) / MAX_IMAGE_ID
    return image_id1, image_id2

def get_keypoints(cursor, img_id):
    dtype=np.float32
    #keypoints = dict(
    #    (image_id, np.frombuffer(data, dtype=dtype).reshape(*shape) )
    #    for image_id, data in cursor.execute(f"SELECT image_id, data FROM keypoints WHERE image_id={image_id}")
    #)
    keypoints = []
    result = cursor.execute(f"SELECT image_id, rows, cols, data FROM keypoints WHERE image_id={img_id}")
    for image_id, rows, cols, data in result:
        shape=(-1, cols)
        coordinates = np.frombuffer(data, dtype=dtype).reshape(*shape)
        keypoints.append({
            "image_id": int(image_id),
            "coordinates": coordinates
        })
    return keypoints

def get_matches(cursor, img1_id = None, img2_id = None):
    dtype=np.uint32
    #matches = dict(
    #    (pair_id_to_image_ids(pair_id), np.frombuffer(data, dtype=dtype).reshape(*shape) )
    #    for pair_id, data in cursor.execute("SELECT pair_id, data FROM matches")
    #)
    matches = []
    for pair_id, rows, cols, data in cursor.execute("SELECT pair_id, rows, cols, data FROM matches"):
        image1, image2 = pair_id_to_image_ids(pair_id)
        image1 = int(image1)
        image2 = int(image2)
        if img2_id and not (image1 == img2_id or image2 == img2_id): continue
        if img1_id and not (image1 == img1_id or image2 == img1_id): continue
        shape=(-1, cols)
        image_matches = np.frombuffer(data, dtype=dtype).reshape(*shape)
        matches.append({
            "image1_id": int(image1),
            "image2_id": int(image2),
            "image_matches": image_matches
        })
    return matches

def get_inlier_matches(cursor, img1_id = None, img2_id = None):
    """
    dtype=np.uint32
    inlier_matches = []
    # get general matches
    for pair_id, rows, cols, data in cursor.execute("SELECT pair_id, rows, cols, data FROM two_view_geometries"):
        image1, image2 = pair_id_to_image_ids(pair_id)
        image1 = int(image1)
        image2 = int(image2)
        if img2_id and not (image1 == img2_id or image2 == img2_id): continue
        if img1_id and not (image1 == img1_id or image2 == img1_id): continue
        shape=(-1, cols)
        image_matches = np.frombuffer(data, dtype=dtype).reshape(*shape)
        inlier_matches.append({
            "image1_id": int(image1),
            "image2_id": int(image2),
            "image_matches": image_matches
        })
    """
    inlier_matches = get_matches(cursor, img1_id, img2_id)
    
    # transform inliers into coordinates
    for im in inlier_matches:
        skipped_kps = []
        #alt_image_matches = np.array() np.reshape(-1, 2)
        alt_image_matches = []
        img1_kps = get_keypoints(cursor, im["image1_id"])[0]
        img2_kps = get_keypoints(cursor, im["image2_id"])[0]
        for im_m in im["image_matches"]:
            img1_kp_index = im_m[0]
            img2_kp_index = im_m[1]
            img1_kp_coords = img1_kps["coordinates"]
            img2_kp_coords = img2_kps["coordinates"]
            img1_kp_coords_len = img1_kp_coords.shape[0]
            img2_kp_coords_len = img2_kp_coords.shape[0]
            if img1_kp_index >= img1_kp_coords_len or img2_kp_index >= img2_kp_coords_len:
                skipped_kps.append((img1_kp_index, img2_kp_index))
                continue
            alt_image_matches.append([
                img1_kp_coords[img1_kp_index],
                img2_kp_coords[img2_kp_index]
            ])
        im["image_matches"] = alt_image_matches
        print(f"skipped matches: {skipped_kps}")
    return inlier_matches

In [ ]:
conn = sqlite3.connect(f"{GROUND_TRUTH_PATH}/database.db")
cursor = conn.cursor()

In [ ]:
# Count the number of features per image
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())
cursor.execute("SELECT * FROM keypoints")
print(result)
print(cursor.fetchall()[0])
#features_per_image = cursor.fetchall()

# Count the total number of matches
#cursor.execute("SELECT COUNT(*) FROM matches WHERE pair_id IN (SELECT id FROM image_pairs)")
#total_matches = cursor.fetchone()[0]

#print("Features per image:", features_per_image)
#print("Total matches:", total_matches)

image_id = 1
img1_kp = get_keypoints(cursor, image_id)[0]
print(len(img1_kp["coordinates"]))
img_matches = get_matches(cursor)
img1_matches = []
for m in img_matches:
    if m["image1_id"] == image_id or m["image2_id"] == image_id:
        img1_matches.append(m)
#print(img1_matches)

In [ ]:
# Select an image by its ID
image1_id = 12
image2_id = 20

# Get the image name and features for the given image ID
image1 = get_image(cursor, image1_id)
image2 = get_image(cursor, image2_id)

matches = get_matches(cursor, image1_id, image2_id)
match = matches[0]

keypoints1 = get_keypoints(cursor, image1_id)[0]
for i, kpc in enumerate(keypoints1["coordinates"]):
    x = kpc[0]
    y = kpc[1]
    cv2.circle(image1, (int(x), int(y)), radius=2, color=(0,  0,  255), thickness=-1)

keypoints2 = get_keypoints(cursor, image2_id)[0]
for i, kpc in enumerate(keypoints2["coordinates"]):
    x = kpc[0]
    y = kpc[1]
    cv2.circle(image2, (int(x), int(y)), radius=2, color=(0,  0,  255), thickness=-1)

RGB_img1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
RGB_img2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)

plt.rcParams["figure.figsize"] = (20,20)
f, axarr = plt.subplots(1,2)
#plt.imshow(RGB_img)
axarr[0].imshow(RGB_img1)
axarr[0].set_title(f"SfM Image #{image1_id}")
axarr[1].imshow(RGB_img2)
axarr[1].set_title(f"SfM Image #{image1_id}")

plt.show()

# Show the image with features
#cv2.imshow("Image with Features", image)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [ ]:
from matplotlib.patches import ConnectionPatch

DISPLAY_CONNECT_LINES = True

inlier_matches = get_inlier_matches(cursor, image1_id, image2_id)[0]
for i, kpc in enumerate(inlier_matches["image_matches"]):
    kpc1 = kpc[0]
    kpc2 = kpc[1]
    kpc1_x = kpc1[0]
    kpc1_y = kpc1[1]
    kpc2_x = kpc2[0]
    kpc2_y = kpc2[1]
    color = (0,  255,  0)
    radius = 3
    cv2.circle(image1, (int(kpc1_x), int(kpc1_y)), radius=radius, color=color, thickness=-1)
    cv2.circle(image2, (int(kpc2_x), int(kpc2_y)), radius=radius, color=color, thickness=-1)

plt.rcParams["figure.figsize"] = (20,20)
fig, axarr = plt.subplots(1,2)
#plt.imshow(RGB_img)
axarr[0].imshow(RGB_img1)
axarr[0].set_title(f"SfM Image #{image1_id}")
#axarr[0].axis('off')
axarr[1].imshow(RGB_img2)
axarr[1].set_title(f"SfM Image #{image1_id}")
#axarr[1].axis('off')

# Loop through each keypoint pair and draw lines
if DISPLAY_CONNECT_LINES:
    for i, kpc in enumerate(inlier_matches["image_matches"]):
        kpc1 = kpc[0]
        kpc2 = kpc[1]
        kpc1_x = kpc1[0]
        kpc1_y = kpc1[1]
        kpc2_x = kpc2[0]
        kpc2_y = kpc2[1]
        
        # Create a connection patch between the two subplots
        con = ConnectionPatch(xyA=(kpc1_x, kpc1_y), xyB=(kpc2_x, kpc2_y),
                                coordsA="data", coordsB="data",
                                axesA=axarr[0], axesB=axarr[1], color="lime")
        # Add the connection patch to the figure, not the axes
        fig.add_artist(con)

plt.show()